# Mosaic AI Agent Framework: Author and deploy a multi-agent system with Genie

This notebook demonstrates how to build a multi-agent system using Mosaic AI Agent Framework and [LangGraph](https://blog.langchain.dev/langgraph-multi-agent-workflows/), where [Genie](https://www.databricks.com/product/ai-bi/genie) is one of the agents.
In this notebook, you:
1. Author a multi-agent system using LangGraph.
1. Wrap the LangGraph agent with MLflow `ChatAgent` to ensure compatibility with Databricks features.
1. Manually test the multi-agent system's output.
1. Log and deploy the multi-agent system.

This example is based on [LangGraph documentation - Multi-agent supervisor example](https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/multi_agent/agent_supervisor.md)

## Why use a Genie agent?

Multi-agent systems consist of multiple AI agents working together, each with specialized capabilities. As one of those agents, Genie allows users to interact with their structured data using natural language.

Unlike SQL functions which can only run pre-defined queries, Genie has the flexibility to create novel queries to answer user questions.

## Prerequisites

- Address all `TODO`s in this notebook.
- Create a Genie Space, see Databricks documentation ([AWS](https://docs.databricks.com/aws/genie/set-up) | [Azure](https://learn.microsoft.com/azure/databricks/genie/set-up)).

 fixing the greeting msg

In [0]:
%pip install -U -qqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv


In [0]:
# pyspark and databricks-connect cannot be installed at the same time. To use databricks-connect, uninstall databricks-connect & pyspark by running 'pip uninstall -y databricks-connect pyspark pyspark-connect' followed by a re-installation of databricks-connect

In [0]:
# %pip install  "databricks-connect>=16.1.1,<16.4" mlflow==3.3.1 langgraph==0.3.4


In [0]:
%pip uninstall -y "databricks-connect>=16.1.1,<16.4" pyspark pyspark-connect

In [0]:
%pip install databricks-connect

In [0]:
dbutils.library.restartPython()


## Define the multi-agent system

Create a multi-agent system in LangGraph using a supervisor agent node directing the following agent nodes:
- **GenieAgent**: The Genie agent that queries and reasons over structured data.
- **Tool-calling agent**: An agent that calls Unity Catalog function tools.

In this example, the tool-calling agent uses the built-in Unity Catalog function `system.ai.python_exec` to execute Python code.
For examples of other tools you can add to your agents, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool)).


#### Wrap the LangGraph agent using the `ChatAgent` interface

Databricks recommends using `ChatAgent` to ensure compatibility with Databricks AI features and to simplify authoring multi-turn conversational agents using an open source standard. 

The `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent.

See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).

#### Write agent code to file

Define the agent code in a single cell below. This lets you write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.


In [0]:
%%writefile agent.py
import functools
import os
from typing import Any, Generator, Literal, Optional

import mlflow
from databricks.sdk import WorkspaceClient
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    DatabricksFunctionClient,
    set_uc_function_client
)
client = DatabricksFunctionClient()
set_uc_function_client(client) 
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import create_react_agent
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from pydantic import BaseModel

###################################################
## Create a GenieAgent with access to a Genie Space
###################################################

# TODO add GENIE_SPACE_ID and a description for this space
# You can find the ID in the URL of the genie room /genie/rooms/<GENIE_SPACE_ID>
# Example description: This Genie agent can answer questions based on a database containing tables related to enterprise software sales, including accounts, opportunities, opportunity history, fiscal periods, quotas, targets, teams, and users. Use Genie to fetch and analyze data from these tables by specifying the relevant columns and filters. Genie can execute SQL queries to provide precise data insights based on your questions.

CRM_GENIE_SPACE_ID = "01f07dbf9d7b120088aaa3248dda8af3"
tm_crm_genie_description = "This Genie agent can answer questions related to  tata motor leads information, their needs in terms of car specifications, budget, time to buy, interactions, notes, feedback on test drive etc. "

tm_crm_genie = GenieAgent(
    genie_space_id=CRM_GENIE_SPACE_ID,
    genie_agent_name="Content-Specialist",
    description=tm_crm_genie_description,
    client=WorkspaceClient(
        host=os.getenv("DB_MODEL_SERVING_HOST_URL"),
        token=os.getenv("DATABRICKS_GENIE_PAT"),
    ),
)


SUMMARIZER_GENIE_SPACE_ID = "01f07def9d5a173ea89f1d37c990e293"
tm_summarizer_genie_description = "This Genie agent can answer questions related to  meetings, opportunities, leads, interactions information."

tm_summarizer_genie = GenieAgent(
    genie_space_id=SUMMARIZER_GENIE_SPACE_ID,
    genie_agent_name="Executive-Assistant",
    description=tm_summarizer_genie_description,
    client=WorkspaceClient(
        host=os.getenv("DB_MODEL_SERVING_HOST_URL"),
        token=os.getenv("DATABRICKS_GENIE_PAT"),
    ),
)

MARKET_RESEARCH_GENIE_SPACE_ID = "01f07f0c53011bbc81f09eec72b146a7"
tm_market_research_genie_description = "This Genie agent can answer questions related to news updates from market which can impact the personalized campaign or sales proposal or any action item for meetings ."

tm_market_research_genie = GenieAgent(
    genie_space_id=MARKET_RESEARCH_GENIE_SPACE_ID,
    genie_agent_name="Market-Researcher",
    description=tm_market_research_genie_description,
    client=WorkspaceClient(
        host=os.getenv("DB_MODEL_SERVING_HOST_URL"),
        token=os.getenv("DATABRICKS_GENIE_PAT"),
    ),
)


############################################
# Define your LLM endpoint and system prompt
############################################

# TODO: Replace with your model serving endpoint
# multi-agent Genie works best with claude 3.7 or gpt 4o models.
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)


############################################################
# Create a code agent
# You can also create agents with access to additional tools
############################################################
tools = []

# TODO if desired, add additional tools and update the description of this agent
# uc_tool_names = ["system.ai.*"]
# uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
# tools.extend(uc_toolkit.tools)
# code_agent_description = (
#     "The Coder agent specializes in solving programming challenges, generating code snippets, debugging issues, and explaining complex coding concepts.",
# )
# code_agent = create_react_agent(llm, tools=tools)

#############################
# Define the supervisor agent
#############################

# TODO update the max number of iterations between supervisor and worker nodes
# before returning to the user
MAX_ITERATIONS = 3

worker_descriptions = {
    "Content-Specialist": tm_crm_genie_description,
    "Executive-Assistant": tm_summarizer_genie_description,
}

research_worker_descriptions = {

"Market-Researcher": tm_market_research_genie_description,
}

formatted_descriptions = "\n".join(
    f"- {name}: {desc}" for name, desc in worker_descriptions.items()
)

supervisor_prompt= """

You are a Sales Co-pilot acting as supervisor, your task is to  understand the intent of the query/request and route the query/request to the respective specialized agents as listed below  :

1. Content-Specialist :  WIll be called if asked for Personalized Outreach & Content Generation for a lead. Supervisor agent will use the information provided by Content-Specialist and generate draft personalized campaigns, emails, messages, and  sales proposals based on leads/prospect/customer data, previous interactions, and vehicle configurations. 

- Input -  lead name (lead, prospect, customer are used interchangeably) or lead id.
You can also answer questions related to leads, their requirements  when asked.

- Gnerate Output :
    Highlight ->
    • Key spec that matches the budget,
    • Available on‑road price with current offers,
    • The interactions, notes and feedback of the leads

- Then call Market-Researcher. To to check market news/updates and impact on the personalized campaign or sales proposal.

Instruction:
    Alwyas check the Market-Researcher agent output before generating the final output and check if the final report will have any impact and modify accordingly.
    Maintain Tata Motors brand tone: friendly, knowledgeable, aspirational.  
    Return ONLY the final email body (no extra commentary) in plain text.


2. Executive-Assistant - Will be called if asked for Meeting Preparation & Summarization , action itrems. supervisor agent will use the information provided by Executive-Assistant and generate  quick summary of all meetings and intercations happened with the lead , generate the action items along with  and opportunity records, recent changes, and relevant news.
- Input : lead name 

- Output :  
    Highlight ->
•       Previous Meeting Summary:
            Give chronological summary of all the meetings & interactions eg- First Contact, Test Drive Meeting, Configuration Discussion, Most recent meeting etc.
•       Current Action Items:
            list the action items in this section

- Then call Market-Researcher. To to check market news/updates and impact on the meeting and action items.

Instruction:
    Alwyas check the Market-Researcher agent output before generating the final output and check if the final report will have any impact and modify accordingly.
    Maintain Tata Motors brand tone: friendly, knowledgeable, aspirational.  
    

Decide between routing between the following workers or ending the conversation if an final answer is provided. Call Market-Researcher agent before generating final answer and incorporare relevant changes in final answer. \n
"""
system_prompt = f"{supervisor_prompt}{formatted_descriptions}"


options = ["FINISH"] + list(worker_descriptions.keys())
FINISH = {"next_node": "FINISH"}

def supervisor_agent(state):
    count = state.get("iteration_count", 0) + 1
    if count > MAX_ITERATIONS:
        return FINISH
    
    class nextNode(BaseModel):
        next_node: Literal[tuple(options)]

    preprocessor = RunnableLambda(
        lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
    )
    supervisor_chain = preprocessor | llm.with_structured_output(nextNode)
    next_node = supervisor_chain.invoke(state).next_node
    
    # if routed back to the same node, exit the loop
    if state.get("next_node") == next_node:
        return FINISH
    return {
        "iteration_count": count,
        "next_node": next_node
    }

#######################################
# Define our multiagent graph structure
#######################################


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": name,
            }
        ]
    }


def final_answer(state):
    prompt = "Using  the content in the messages, respond to the previous user question using the answer given by the other assistant messages. If you do not find answer given by the other assistant messages do not mention that, instead clarify the questions if you have doubt. If sales rep/user says Hello or Hi that is starting cnversation, alway Greet and ask - How can I help you. "
    preprocessor = RunnableLambda(
        lambda state: state["messages"] + [{"role": "user", "content": prompt}]
    )
    final_answer_chain = preprocessor | llm
    return {"messages": [final_answer_chain.invoke(state)]}


class AgentState(ChatAgentState):
    next_node: str
    iteration_count: int


tm_summarizer_genie_node = functools.partial(agent_node, agent=tm_summarizer_genie, name="Executive-Assistant")
tm_crm_genie_node = functools.partial(agent_node, agent=tm_crm_genie, name="Content-Specialist")
tm_market_research_genie_node = functools.partial(agent_node, agent=tm_market_research_genie, name="Market-Researcher")

workflow = StateGraph(AgentState)
workflow.add_node("Content-Specialist", tm_crm_genie_node)
workflow.add_node("Executive-Assistant", tm_summarizer_genie_node)
workflow.add_node("Market-Researcher", tm_market_research_genie_node)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("final_answer", final_answer)

workflow.set_entry_point("supervisor")
# We want our workers to ALWAYS "check back" to "Market-Researcher" then go back to the supervisor when done
for worker in worker_descriptions.keys():
    workflow.add_edge(worker, "Market-Researcher")
    workflow.add_edge("Market-Researcher", "supervisor")
    

# Let the supervisor decide which next node to go
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next_node"],
    {**{k: k for k in worker_descriptions.keys()}, "FINISH": "final_answer"},
)
workflow.add_edge("final_answer", END)
multi_agent = workflow.compile()

###################################
# Wrap our multi-agent in ChatAgent
###################################


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg})
                    for msg in node_data.get("messages", [])
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
mlflow.langchain.autolog()
AGENT = LangGraphChatAgent(multi_agent)
mlflow.models.set_model(AGENT)

## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

**TODO**: Replace this placeholder `input_example` with a domain-specific prompt for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import AGENT

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "Hello"}]})

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "can you genearate the sales proposal for Priya Iyer?"}]})

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "give me the meeting summary and action items for Rohit Sharma"}]})

## Create a Personal Access Token (PAT) as a Databricks secret
In order to access the Genie Space and its underlying resources, we need to create a PAT
- This can either be your own PAT or that of a System Principal ([AWS](https://docs.databricks.com/aws/en/dev-tools/auth/oauth-m2m) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/auth/oauth-m2m)). You will have to rotate this token yourself upon expiry.
- Add secrets-based environment variables to a model serving endpoint ([AWS](https://docs.databricks.com/aws/en/machine-learning/model-serving/store-env-variable-model-serving#add-secrets-based-environment-variables) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/store-env-variable-model-serving#add-secrets-based-environment-variables)).
- You can reference the table in the deploy docs for the right permissions level for each resource: ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/deploy-agent#automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/deploy-agent#automatic-authentication-passthrough)).
  - Provision with `CAN RUN` on the Genie Space
  - Provision with `CAN USE` on the SQL Warehouse powering the Genie Space
  - Provision with `SELECT` on underlying Unity Catalog Tables 
  - Provision with `EXECUTE` on underyling Unity Catalog Functions 

In [0]:
import os
from dbruntime.databricks_repl_context import get_context

# TODO: set WORKSPACE_URL manually if it cannot be inferred from the current notebook
WORKSPACE_URL = None
WORKSPACE_URL = "https://adb-984752964297111.11.azuredatabricks.net/"
if WORKSPACE_URL is None:
  workspace_url_hostname = get_context().workspaceUrl
  assert workspace_url_hostname is not None, "Unable to look up current workspace URL. This can happen if running against serverless compute. Manually set WORKSPACE_URL yourself above, or run this notebook against classic compute"
  WORKSPACE_URL = "https://adb-984752964297111.11.azuredatabricks.net/"
  
  #f"https://{workspace_url_hostname}"

In [0]:
WORKSPACE_URL

In [0]:
# TODO: set secret_scope_name and secret_key_name to access your PAT
# secret_scope_name = ""
# secret_key_name = ""

os.environ["DB_MODEL_SERVING_HOST_URL"] = WORKSPACE_URL
assert os.environ["DB_MODEL_SERVING_HOST_URL"] is not None

#TODO: set DATABRICKS_GENIE_PAT to your PAT
os.environ["DATABRICKS_GENIE_PAT"] = "YOUR_PAT_TOKEN"

# os.environ["DATABRICKS_GENIE_PAT"] = dbutils.secrets.get(
#     scope=secret_scope_name, key=secret_key_name
# )
assert os.environ["DATABRICKS_GENIE_PAT"] is not None, (
    "The DATABRICKS_GENIE_PAT was not properly set to the PAT secret"
)

In [0]:
from agent import AGENT, tm_crm_genie_description, tm_summarizer_genie_description, tm_market_research_genie_description

# assert tm_summarizer_genie_description != "This Genie agent can answer questions related to  meetings, opportunities, leads, interactions information.", (
#     "Remember to update the tm_summarizer_genie agent description for higher quality answers."
# )


# assert tm_crm_genie_description != "This Genie agent can answer questions related to  tata motor leads information, their needs in terms of car specifications, budget, time to buy, interactions, notes, feedback on test drive etc. ", (
#     "Remember to update the tm_crm_genie  agent description for higher quality answers."
# )

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "generate a sales proposal for Rohit Sharma?",
        }
    ]
}
#AGENT.predict(input_example)

In [0]:
for event in AGENT.predict_stream(input_example):
  print(event, "-----------\n")

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`
  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/log-agent#resources) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).

  - **TODO**: If the SQL Warehouse powering your Genie space has secured permissions, include the warehouse ID and table name in your resources to enable passthrough authentication. ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/log-agent#-specify-resources-for-automatic-authentication-passthrough-system-authentication) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#-specify-resources-for-automatic-authentication-passthrough-system-authentication)).

In [0]:
from pkg_resources import get_distribution

version = get_distribution('databricks-connect').version
print(f"databricks-connect=={version}")



version = get_distribution('databricks-langchain').version
print(f"ddatabricks-langchain=={version}")


version = get_distribution('langgraph').version
print(f"langgraph=={version}")


In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from agent import  LLM_ENDPOINT_NAME, tools, CRM_GENIE_SPACE_ID , SUMMARIZER_GENIE_SPACE_ID
from databricks_langchain import UnityCatalogTool, VectorSearchRetrieverTool
from mlflow.models.resources import (
    DatabricksFunction,
    DatabricksGenieSpace,
    DatabricksServingEndpoint,
#   DatabricksSQLWarehouse,
    DatabricksTable,
)
from pkg_resources import get_distribution


CRM_GENIE_SPACE_ID = "01f07dbf9d7b120088aaa3248dda8af3"
SUMMARIZER_GENIE_SPACE_ID = "01f07def9d5a173ea89f1d37c990e293"
MARKET_RESEARCH_GENIE_SPACE_ID = "01f07f0c53011bbc81f09eec72b146a7"

# TODO: Manually include underlying resources if needed. See the TODO in the markdown above for more information.
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksGenieSpace(genie_space_id=CRM_GENIE_SPACE_ID),
    DatabricksGenieSpace(genie_space_id=SUMMARIZER_GENIE_SPACE_ID),
    DatabricksGenieSpace(genie_space_id=MARKET_RESEARCH_GENIE_SPACE_ID),
#   DatabricksSQLWarehouse(warehouse_id="your_warehouse_id"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.carspec"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.interaction"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.leads"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.meetings"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.oppertunity"),
    DatabricksTable(table_name="sarbanimaiti_catalog.auto_data.tm_news")
]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        pip_requirements=[
            f"databricks-connect>=16.1.1,<16.4",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
    )

In [0]:
resources

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data=input_example,
    env_manager="uv",
)

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model, fixing the greetiung msg
catalog = "sarbanimaiti_catalog"
schema = "auto_data"
model_name = "tm_sales_copilotV3"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

## Deploy the agent

In [0]:
from databricks import agents

# agents.deploy(
#     UC_MODEL_NAME,
#     uc_registered_model_info.version,
#     tags={"endpointSource": "docs"},
#     environment_vars={
#         "DATABRICKS_GENIE_PAT": f"{{{{secrets/{secret_scope_name}/{secret_key_name}}}}}"
#     },
# )
#os.environ["DATABRICKS_GENIE_PAT"] = "YOUR_PAT_TOKEN"

#TODO change DATABRICKS_GENIE_PAT" to "YOUR_PAT_TOKEN"
agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "docs"},
    environment_vars={
        "DATABRICKS_GENIE_PAT": "YOUR_PAT_TOKEN"
    },
)

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).